## 0. Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from datetime import datetime

## 1. Parsing data

In [2]:
# make sure that data.csv file is downloaded
df = pd.read_csv('data.csv')
df = df.dropna(subset=['contracts'])
df.head()

,id,application_date,contracts
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
5,2925215.0,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
6,2925216.0,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
8,2925218.0,2024-02-12 19:40:15.507000+00:00,"[{""contract_id"": """", ""bank"": ""062"", ""summa"": ""..."


As we can see `contracts` column is json format and we need to parse it.

Let's learn how the types of `contracts` look like

In [3]:
types = set()
for row in range(df.shape[0]):
  types.add(type(json.loads(df.iloc[row]['contracts'])))
types

{dict, list}

So, in some rows, contracts consist of multiple contracts, in some cases it is just one contract (hence they are `dict` type)

In [4]:
sample = json.loads(df.iloc[0]['contracts'])
keys = [key for key in sample[0]]
keys

['contract_id',
 'bank',
 'summa',
 'loan_summa',
 'claim_date',
 'claim_id',
 'contract_date']

In [5]:
df['contracts']

1      [{"contract_id": 522530, "bank": "003", "summa...
3      [{"contract_id": 522530, "bank": "003", "summa...
5      [{"contract_id": 522530, "bank": "003", "summa...
6      [{"contract_id": 522530, "bank": "003", "summa...
8      [{"contract_id": "", "bank": "062", "summa": "...
                             ...                        
985    [{"contract_id": "", "bank": "014", "summa": "...
986    [{"contract_id": "", "bank": "014", "summa": "...
989    [{"contract_id": 18410, "bank": "004", "summa"...
991    [{"contract_id": 18410, "bank": "004", "summa"...
995    [{"contract_id": 18410, "bank": "004", "summa"...
Name: contracts, Length: 495, dtype: object

In [6]:
df['contracts'] = df['contracts'].apply(lambda x: json.loads(x))

We can see there are a lot of duplicates.
First 4 contracts are literally the same

In [7]:
df = df.drop_duplicates(subset=['contracts'])

In [9]:
df['contracts'].head()

1     [{'contract_id': 522530, 'bank': '003', 'summa...
8     [{'contract_id': '', 'bank': '062', 'summa': '...
21    [{'contract_id': '', 'bank': '014', 'summa': '...
24    [{'contract_id': '', 'bank': '014', 'summa': '...
33    [{'contract_id': 19795, 'bank': '004', 'summa'...
Name: contracts, dtype: object

We can see that, we don't have duplicates anymore

In some parts of feature engineering, we might be interested how
values look like in the contracts

In [10]:
def val_count(col):
  c = Counter()
  for row in range(df.shape[0]):
    if isinstance(df.iloc[row]['contracts'], dict):
      if col in df.iloc[row]['contracts']:
        c[df.iloc[row]['contracts'][col]] += 1
      else:
        c[None] += 1
    else:
      for contract in df.iloc[row]['contracts']:
        if col in contract:
          c[contract[col]] += 1
        else:
          c[None] += 1
  return c

## 2. Feature `tot_claim_cnt_l180d`

Description: number of claims for last 180 days
Source: contracts
Key fields: `claim_id`, `claim_date`
Special notes:
1. In case `claim date` is `null`, don't take into consideration such claims.
2. In case no claims, then put -3 as a value of this feature.

In [11]:
current_date = datetime.now()
date_format = '%d.%m.%Y'

# this part of the code developed at 11th June, so if you want to reproduce same results,
# please change current_date part to the  `datetime.strptime('11.06.2024', date_format)`

def less_than_180d(date):
  if date is None:
    return 0
  given_date = datetime.strptime(date, date_format)

  difference = current_date - given_date
  return int(difference.days <= 180)

In [12]:
assert less_than_180d('11.06.2024') == 1
assert less_than_180d(None) == 0


In [ ]:
claim_id_vals = val_count('claim_id')
claim_id_vals

In [16]:
def tot_claim_l180d(contracts):
  if isinstance(contracts, dict):
    return less_than_180d(contracts['claim_date']) if less_than_180d(contracts['claim_date']) else -3
  else:
    claim_ids = set()
    for contract in contracts:
      if less_than_180d(contract['claim_date']):
        claim_ids.add(contract['claim_id'])
    return len(claim_ids) if len(claim_ids) else -3

In [17]:
df['tot_claim_cnt_l180d'] = df['contracts'].apply(tot_claim_l180d)

In [18]:
# let's check if there invalid values
df['tot_claim_cnt_l180d'].value_counts()

tot_claim_cnt_l180d
-3     37
 2     29
 1     22
 3     11
 5      7
 4      6
 11     4
 9      3
 6      3
 13     3
 8      2
 7      2
 12     1
 34     1
 21     1
 44     1
 10     1
 60     1
 55     1
 18     1
Name: count, dtype: int64

## 3. Feature `disb_bank_loan_wo_tbc`

Description: Sum of exposue of loans without TBC loans. Exposure means here field `loan_summa`.
Source: `contracts`
Key fields: `bank`, `loan_summa`, `contract_date`
Special notes:
1. Consider only loans where field `bank` is not in `['LIZ', 'LOM', 'MKO', 'SUG', null]`.
2. Disbursed loans means loans where `contract_date` is not `null`

In case no claims, then put -3 as a value of this feature.

If no loans at all, then put -1 as a value of this feature.

In [19]:
# let's check how the names of banks look like
banks_count = val_count('bank')
banks_count

Counter({'003': 34,
         '014': 206,
         '053': 35,
         '062': 395,
         '009': 59,
         '063': 493,
         '055': 10,
         'MKO': 89,
         '002': 76,
         '013': 89,
         '004': 218,
         None: 37,
         'ORG': 42,
         '011': 37,
         '012': 42,
         '006': 62,
         '051': 27,
         '030': 72,
         '033': 181,
         '005': 24,
         '061': 78,
         'LOM': 205,
         '049': 16,
         '057': 3,
         '066': 1,
         '031': 15,
         '008': 5,
         '048': 5})

"In case no claims, then put -3 as a value of this feature.
If no loans at all, then put -1 as a value of this feature"
This part was confusing little bit to me, so this is how I
decided what record have no claims and what record has no loans at all:
1. If none of the contracts has valid `contract_date`, then this contract has no claims
2. If the total sum of `loan_summa` is 0, then it counted as no loans at all
Depending on the definiton it might change.

In [20]:
not_allowed_banks = ['LIZ', 'LOM', 'MKO', 'SUG', None]

def valid_date(date):
  return date is not None and date.strip() != ''

def valid_loan(contract):
  return 'bank' in contract and contract['bank'] not in not_allowed_banks and isinstance(contract['loan_summa'], int)

In [21]:
def calculated_loan(contract):
  claim = loan = summa = 0
  # here I added `contract['contract_date'].strip() != ''`
  # since some dates are empty strings and they pass the first check
  if valid_date(contract['contract_date']):
    claim = 1
    # here I added 'bank' in contract checking,
    # since some contracts do not have this property
    if valid_loan(contract):
      loan = 1
      summa = contract['loan_summa']
  return claim, loan, summa

def sum_of_exposure_loans(contracts):
  total = 0
  claim = loan = 0
  if isinstance(contracts, dict):
    claim, loan, summa = calculated_loan(contracts)
    if claim == 0:
      return -3
    if loan == 0:
      return -1
    return summa if summa else -1
  else:
    for contract in contracts:
      claim_, loan_, summa = calculated_loan(contract)
      claim |= claim_
      loan |= loan_
      total += summa
    if claim == 0:
      return -3
    elif loan == 0:
      return -1
  return total if total != 0 else -1

As we can see some `loan_summa` quantities are equal to empty string.

In [22]:
df['disb_bank_loan_wo_tbc'] = df['contracts'].apply(sum_of_exposure_loans)

In [23]:
# let's check if there invalid values
df['disb_bank_loan_wo_tbc'].value_counts()

disb_bank_loan_wo_tbc
-1             39
-3             28
 1148778676     1
 3319227081     1
 491809096      1
               ..
 3692414810     1
 7045370606     1
 431182433      1
 1374999700     1
 217320883      1
Name: count, Length: 72, dtype: int64

## 4. Feature `day_sinlastloan`


Description: Number of days since last loan.

Source: `contracts`

Key fields: `contract_date`, `summa`

Special notes:
1. Take last loan of client where `summa` is not `null` and calculate number of days from `contract_date` of this loan to `application date`.

In case no claims at all, then put `-3` as a value of this feature.

In case no loans at all, then put `-1` as a value of this feature.

In [ ]:
vals_loan = val_count('summa')
vals_loan

As we can see some `loan_summa` values are `0` or empty string.

In [25]:
def valid_loan_summa(summa):
  return isinstance(summa, int)

In [26]:
# given_date = datetime.strptime(date, date_format)
date_format = '%d.%m.%Y'
def last_loan(contracts):
  last_loan = -1
  claim = loan = 0
  if isinstance(contracts, dict):
    if not valid_date(contracts['contract_date']):
      return -3
    if not valid_loan_summa(contracts['summa']):
      return -1
    return datetime.strptime(contracts['contract_date'], date_format)
  for contract in contracts:
    if not valid_date(contract['contract_date']):
      continue
    claim |= 1
    if valid_loan_summa(contract['summa']):
      given_date = datetime.strptime(contract['contract_date'], date_format)
      loan |= 1
      if last_loan == -1 or given_date > last_loan:
        last_loan = given_date
  if claim == 0:
    return -3
  if loan == 0:
    return -1
  return last_loan

In [27]:
def diff_contract_application(row):
  last_loan, application_date = row['last_loan'], row['application_date']
  if last_loan == -3:
    return -3
  if last_loan == -1:
    return -1
  application_date = datetime.strptime(application_date[:10], "%Y-%m-%d")
  return (application_date - last_loan).days

In [28]:
df['application_date'].value_counts()

application_date
2024-02-12 19:24:29.135000+00:00    1
2024-02-13 05:43:04.731000+00:00    1
2024-02-13 05:37:40.622000+00:00    1
2024-02-13 05:39:57.399000+00:00    1
2024-02-13 05:40:30.287000+00:00    1
                                   ..
2024-02-13 04:54:49.641000+00:00    1
2024-02-13 04:54:10.776000+00:00    1
2024-02-13 04:53:41.149000+00:00    1
2024-02-13 04:52:42.959000+00:00    1
2024-02-13 06:09:26.994000+00:00    1
Name: count, Length: 137, dtype: int64

In [29]:
df['last_loan'] = df['contracts'].apply(last_loan)

In [30]:
df['day_sinlastloan'] = df.apply(lambda x: diff_contract_application(x), axis=1)

Let's check the values and if there any invalid ones:

In [31]:
df['day_sinlastloan'].value_counts()

day_sinlastloan
-3      28
 15      4
 4       3
 309     3
 13      3
        ..
 106     1
 553     1
 156     1
 369     1
 729     1
Name: count, Length: 95, dtype: int64

In [32]:
df.columns

Index(['id', 'application_date', 'contracts', 'tot_claim_cnt_l180d',
       'disb_bank_loan_wo_tbc', 'last_loan', 'day_sinlastloan'],
      dtype='object')

In [33]:
# last_loan column is only needed to calculate day_sinlastloan,
# so, let's drop it
df = df.drop(columns=['last_loan'], axis=1)

In [34]:
df.columns

Index(['id', 'application_date', 'contracts', 'tot_claim_cnt_l180d',
       'disb_bank_loan_wo_tbc', 'day_sinlastloan'],
      dtype='object')

## 5. Creating dataset from features


In [35]:
contract_features_cols = ['tot_claim_cnt_l180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']
contract_features = df[contract_features_cols]

In [36]:
contract_features.to_csv('contract_features.csv', index=False)

In [37]:
con_fet = pd.read_csv('contract_features.csv')
con_fet.head()

,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,44,-1,427
1,13,-1,288
2,9,2537851751,170
3,3,-3,-3
4,5,559300000,158
